In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [3]:
with open("../dataset/cifar10_train.pkl", "rb") as f:
    x_train, y_train = pkl.load(f)
with open("../dataset/cifar10_test.pkl", "rb") as f:
    x_test, y_test = pkl.load(f)

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [5]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [31]:
val_split = int((1-VAL_SPLIT)*len(x_train))
x_train , x_val = x_train[:val_split], x_train[val_split:]
y_train, y_val = y_train[:val_split], y_train[val_split:]

# Define data augmentations

In [32]:
datagen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                             zoom_range=.2,
                             horizontal_flip=True, 
                             vertical_flip=True)

# Build model

In [33]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [34]:
base_model = Xception(include_top=False, input_tensor=input_tensor, weights=None)

In [35]:
base_model.load_weights('../pretrained_weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [36]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [37]:
xception = Model(inputs=base_model.input, outputs=predictions)

In [38]:
xception.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [40]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [41]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=True, write_images=False)

In [42]:
xception.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                        steps_per_epoch=len(x_train) / BATCH_SIZE,
                        validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                        validation_steps=len(x_val) / BATCH_SIZE,
                        epochs=EPOCHS, 
                        callbacks=[tensorboard_callback], 
                        verbose=1)

Epoch 1/100
79/78 [==============================] - 58s 737ms/step - loss: 2.1623 - acc: 0.1780 - val_loss: 2.2387 - val_acc: 0.2114
Epoch 2/100
79/78 [==============================] - 47s 595ms/step - loss: 1.8125 - acc: 0.3381 - val_loss: 1.9173 - val_acc: 0.3048
Epoch 3/100
79/78 [==============================] - 46s 589ms/step - loss: 1.4789 - acc: 0.4978 - val_loss: 1.4590 - val_acc: 0.5247
Epoch 4/100
79/78 [==============================] - 46s 585ms/step - loss: 1.1987 - acc: 0.5788 - val_loss: 1.2419 - val_acc: 0.5704
Epoch 5/100
79/78 [==============================] - 46s 587ms/step - loss: 1.0931 - acc: 0.6157 - val_loss: 1.1104 - val_acc: 0.6068
Epoch 6/100
79/78 [==============================] - 46s 582ms/step - loss: 1.0296 - acc: 0.6357 - val_loss: 1.0828 - val_acc: 0.6192
Epoch 7/100
79/78 [==============================] - 47s 591ms/step - loss: 0.9687 - acc: 0.6595 - val_loss: 1.0504 - val_acc: 0.6262
Epoch 8/100
79/78 [==============================] - 47s 590ms

Epoch 62/100
79/78 [==============================] - 46s 587ms/step - loss: 0.2961 - acc: 0.8946 - val_loss: 0.8006 - val_acc: 0.7735
Epoch 63/100
79/78 [==============================] - 46s 586ms/step - loss: 0.2985 - acc: 0.8950 - val_loss: 0.7801 - val_acc: 0.7689
Epoch 64/100
79/78 [==============================] - 46s 587ms/step - loss: 0.2821 - acc: 0.9003 - val_loss: 0.8079 - val_acc: 0.7666
Epoch 65/100
79/78 [==============================] - 46s 588ms/step - loss: 0.2851 - acc: 0.9000 - val_loss: 0.8065 - val_acc: 0.7710
Epoch 66/100
79/78 [==============================] - 47s 589ms/step - loss: 0.2745 - acc: 0.9033 - val_loss: 0.8076 - val_acc: 0.7655
Epoch 67/100
79/78 [==============================] - 46s 586ms/step - loss: 0.2735 - acc: 0.9029 - val_loss: 0.8141 - val_acc: 0.7712
Epoch 68/100
79/78 [==============================] - 46s 587ms/step - loss: 0.2701 - acc: 0.9054 - val_loss: 0.8278 - val_acc: 0.7626
Epoch 69/100
79/78 [==============================] - 4

In [43]:
xception.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 10s 962us/step


[0.8776747555255889, 0.8061]